In [ ]:
### Notebook for plotting graphics from polygons

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
from bs4 import BeautifulSoup

In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, mapping
import ast

# Read the CSV file
url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/homeplate_outbound.csv"
df = pd.read_csv(url)

# Function to convert string representation of a tuple to a tuple
def string_to_tuple(string):
    return tuple(ast.literal_eval(string))

# Function to convert string representation of a polygon to a polygon
def string_to_polygon(string):
    # Remove "POLYGON Z " from the string and the extra parentheses
    coords_str = string[10:-1].replace('(', '').replace(')', '')
    
    # Convert the string to a list of coordinate tuples, skipping invalid ones
    coords = []
    for coord in coords_str.split(','):
        try:
            coords.append(tuple(map(float, coord.split()))[:2])
        except ValueError:
            print(f"Skipped invalid coordinate: {coord}")
            continue
    
    # Create a Polygon object
    return Polygon(coords)


# Extract 'Homeplate' and 'poly' data
df['homeplate_coords'] = df['homeplate'].apply(string_to_tuple)
df['polygon'] = df['poly'].apply(string_to_polygon)

# Create a GeoDataFrame using the 'polygon' column
gdf = GeoDataFrame(df, geometry='polygon', crs="EPSG:4326")

# Add 'Homeplate' points as a separate GeoDataFrame
homeplate_gdf = gpd.GeoDataFrame(df, geometry=df['homeplate_coords'].apply(Point), crs="EPSG:4326")

# Save the GeoDataFrames as GeoJSON files
gdf.to_file("homeplate_outbound_polygons.geojson", driver="GeoJSON")
homeplate_gdf.to_file("homeplate_outbound_points.geojson", driver="GeoJSON")


ValueError: could not convert string to float: 'ON'